In [ ]:
import credentials as cr
import SQL_requests as rq
import pandas as pd
import warnings
from db_connect import Db_connect
from recommendation_system import Recommendation
from sklearn.metrics.pairwise import cosine_similarity


%load_ext autoreload
%autoreload 2

In [ ]:
# redirect_uri = 'http://localhost/'
data_path = cr.DATA_PATH

db = Db_connect()
rc = Recommendation()

warnings.filterwarnings("ignore")

In [ ]:
db.init_connection()
top_30_titles = pd.read_sql_query(rq.SELECT_PLAYLIST_TITLES + "'6AsIwsN6mH7dQSChLsqSlY'", db.connector).set_index('id')
track_titles = pd.read_sql_query(rq.SELECT_TRACKS_TITLES, db.connector).set_index('id')
track_features = pd.read_sql_query(rq.SELECT_TRACKS_FEATURES, db.connector).set_index('id')
db.close_connection()

## kNearestNeighbors (unsupervised)

### Without data processing

In [ ]:
playlist, nonplaylist = rc.generate_playlist_nonplaylist(track_features, top_30_titles)
recommendation, recommendation_features = rc.get_recommendations(playlist, nonplaylist, track_features, track_titles)
rc.visualize_cover(recommendation)
display(recommendation)

### With data processing

In [ ]:
track_features_processed = rc.process_data(track_features)
display(track_features_processed.head())

playlist, nonplaylist = rc.generate_playlist_nonplaylist(track_features_processed, top_30_titles)
playlist_sum = playlist.sum(axis = 0)

recommendation, recommendation_features = rc.get_recommendations_processed(playlist, nonplaylist, track_features_processed, track_titles)

rc.visualize_cover(recommendation)

display(recommendation)


In [ ]:
cosine = rc.get_cosine_similarity(playlist, recommendation_features, track_titles)
display(cosine)
print("MEAN COSINE SIMILARITY : ", cosine.sim_mean.mean())

## Cosine Similarity

In [ ]:
track_features_processed = rc.process_data(track_features)

playlist, nonplaylist = rc.generate_playlist_nonplaylist(track_features_processed, top_30_titles)
playlist_sum = playlist.sum(axis = 0)

nonplaylist_copy = nonplaylist.copy()

nonplaylist_copy['sim'] = cosine_similarity(nonplaylist.values, playlist_sum.values.reshape(1, -1))[:,0]

recommendation = nonplaylist_copy.sort_values('sim',ascending = False).head(10)
recommendation = recommendation.merge(track_titles, left_index=True, right_index=True)

rc.visualize_cover(recommendation)

recommendation.head()